In [2]:
import pandas as pd
import numpy as np
# import datetime
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

In [3]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

In [4]:
def current_date():
    return str(datetime.now())

In [13]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  return create_tuple(request)
fume_query(target="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat", server="biotech_main", start="2021-12-25T00:00:00.000", end=current_date())

<Response [200]>


array([(40.5, '2021-12-25T05:00:00.000'),
       (40.5, '2021-12-25T05:15:00.000'),
       (40.5, '2021-12-25T05:30:00.000'), ...,
       (35.5, '2023-03-10T21:30:00.000'),
       (35.5, '2023-03-10T21:45:00.000'),
       (35.5, '2023-03-10T22:00:00.000')],
      dtype=[('value', '<f8'), ('ts', '<M8[ms]')])

In [14]:
def get_oat(oat_point, server, start, end):
  oat_master = fume_query(oat_point, server, start, end)

  oat_list = pd.Series(data=[i[0] for i in oat_master], index=[i[1] for i in oat_master])
  oat_list.to_csv("test.csv")
  oat_list = oat_list[~oat_list.index.duplicated()]
  print("\nOAT List: ", oat_list)

  df = pd.concat([oat_list], axis=1)
  df.columns = ["OAT"]
  df = df.dropna()
  display(df)
  df.to_csv("test2.csv")

  df = df.groupby(pd.Grouper(freq='5Min', label='right')).sum()
  return df["OAT"]

get_oat(oat_point="#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

# get_oat(oat_point="#biotech_oa_conditions/temperature", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))


<Response [200]>

OAT List:  2021-11-17 05:00:00    35.2
2021-11-17 05:15:00    35.3
2021-11-17 05:30:00    35.5
2021-11-17 05:45:00    35.6
2021-11-17 06:00:00    35.3
                       ... 
2021-11-18 03:45:00    50.5
2021-11-18 04:00:00    50.3
2021-11-18 04:15:00    50.3
2021-11-18 04:30:00    50.5
2021-11-18 04:45:00    50.6
Length: 96, dtype: float64


,OAT
2021-11-17 05:00:00,35.2
2021-11-17 05:15:00,35.3
2021-11-17 05:30:00,35.5
2021-11-17 05:45:00,35.6
2021-11-17 06:00:00,35.3
...,...
2021-11-18 03:45:00,50.5
2021-11-18 04:00:00,50.3
2021-11-18 04:15:00,50.3
2021-11-18 04:30:00,50.5


2021-11-17 05:05:00    35.2
2021-11-17 05:10:00     0.0
2021-11-17 05:15:00     0.0
2021-11-17 05:20:00    35.3
2021-11-17 05:25:00     0.0
                       ... 
2021-11-18 04:30:00     0.0
2021-11-18 04:35:00    50.5
2021-11-18 04:40:00     0.0
2021-11-18 04:45:00     0.0
2021-11-18 04:50:00    50.6
Freq: 5T, Name: OAT, Length: 286, dtype: float64